In [108]:
import random
import numpy as np
import pandas as pd
from data_process import get_CIFAR10_data
%matplotlib inline
from save_submission import output_submission_csv

In [109]:
cd- C:/Users/Samvidha reddy/Downloads/code_ma

[WinError 123] The filename, directory name, or volume label syntax is incorrect: '- C:/Users/Samvidha reddy/Downloads/code_ma'
C:\Users\Samvidha reddy\Downloads\code_masters


In [110]:
cd code_masters

[WinError 2] The system cannot find the file specified: 'code_masters'
C:\Users\Samvidha reddy\Downloads\code_masters


# Loading CIFAR-10

In the following cells we determine the number of images for each split and load the images.
<br /> 
TRAIN_IMAGES + VAL_IMAGES = (0, 50000]
, TEST_IMAGES = 10000

In [111]:
# You can change these numbers for experimentation
# For submission we will use the default values 
TRAIN_IMAGES = 40000
VAL_IMAGES = 10000

In [112]:
data = get_CIFAR10_data(TRAIN_IMAGES, VAL_IMAGES)
X_train_CIFAR, y_train_CIFAR = data['X_train'], data['y_train']
X_val_CIFAR, y_val_CIFAR = data['X_val'], data['y_val']
X_test_CIFAR, y_test_CIFAR = data['X_test'], data['y_test']
n_class_CIFAR = len(np.unique(y_test_CIFAR))

Convert the sets of images from dimensions of **(N, 3, 32, 32) -> (N, 3072)** where N is the number of images so that each **3x32x32** image is represented by a single vector.

In [113]:
X_train_CIFAR = np.reshape(X_train_CIFAR, (X_train_CIFAR.shape[0], -1))
X_val_CIFAR = np.reshape(X_val_CIFAR, (X_val_CIFAR.shape[0], -1))
X_test_CIFAR = np.reshape(X_test_CIFAR, (X_test_CIFAR.shape[0], -1))

In [114]:
X_train_CIFAR = X_train_CIFAR.T
X_val_CIFAR = X_val_CIFAR.T
X_test_CIFAR = X_test_CIFAR.T
#y_train_CIFAR.shape

# Get Accuracy
This function computes how well your model performs using accuracy as a metric.


In [115]:
def get_acc(pred, y_test):
    return np.sum(y_test==pred)/len(y_test)*100

# Perceptron

Perceptron has 2 hyperparameters that you can experiment with:
- **Learning rate** - controls how much we change the current weights of the classifier during each update. We set it at a default value of 0.5, but you should experiment with different values. We recommend changing the learning rate by factors of 10 and observing how the performance of the classifier changes. You should also try adding a **decay** which slowly reduces the learning rate over each epoch.
- **Number of Epochs** - An epoch is a complete iterative pass over all of the data in the dataset. During an epoch we predict a label using the classifier and then update the weights of the classifier according the perceptron update rule for each sample in the training set. You should try different values for the number of training epochs and report your results.


The following code: 
- Creates an instance of the Perceptron classifier class 
- The train function of the Perceptron class is trained on the training data
- We use the predict function to find the training accuracy as well as the testing accuracy


# Model Perceptron

In [116]:
import numpy as np
import math

m,n = 50000, 3072


class Perceptron:
    def __init__(self, n_class: int, lr: float, epochs: int):
        """Initialize a new classifier.
        Parameters:
            n_class: the number of classes
            lr: the learning rate
            epochs: the number of epochs to train for
        """
        #self.w = ###### TODO: change this
        self.lr = 0.1 ###### TODO: change this
        self.epochs = 500 ###### TODO: change this
        self.n_class = 10 ###### TODO: change this
        
    def init_params(self):
        W1 = np.random.rand(10, 3072) 
        b1 = np.random.rand(10, 1) 
        W2 = np.random.rand(10, 10) 
        b2 = np.random.rand(10, 1)
        return W1, b1, W2, b2
    
    def sigmoid(self, x): 
        x = x / np.linalg.norm(x, axis=-1)[:, np.newaxis]
        return 1 / (1 + np.exp(-x))
   
    def ReLU(self,Z):
        return np.maximum(Z, 0)
    
    def ReLU_deriv(self,Z):
        return Z > 0
    
    def softmax(self, Z):
        Z = Z / np.linalg.norm(Z, axis=-1)[:, np.newaxis]
        #b = x.max()
        #y = np.exp(x - b)
        #return y / y.sum()
        #b = np.max(Z)
        #print(b)
        A = np.exp(Z) / np.sum(np.exp(Z), axis = 0)
        return A
    
    def forward_prop(self, W1, b1, W2, b2, X):
        Z1 = W1.dot(X) + b1
        A1 = self.sigmoid(Z1)
        Z2 = W2.dot(A1) + b2
        A2 = self.softmax(Z2)
        return Z1, A1, Z2, A2
    
    def d_sigmoid(self,x):
        s = self.sigmoid(x)
        return s * (1 - s)
    
    def one_hot(self,Y):
        one_hot_Y = np.zeros((Y.size, Y.max() + 1))
        one_hot_Y[np.arange(Y.size), Y] = 1
        one_hot_Y = one_hot_Y.T
        return one_hot_Y
    
    def backward_prop(self,Z1, A1, Z2, A2, W1, W2, X, Y):
        one_hot_Y = self.one_hot(Y)
        dZ2 = A2 - one_hot_Y
        dW2 = 1 / m * dZ2.dot(A1.T)
        db2 = 1 / m * np.sum(dZ2)
        dZ1 = W2.T.dot(dZ2) * self.d_sigmoid(Z1)
        dW1 = 1 / m * dZ1.dot(X.T)
        db1 = 1 / m * np.sum(dZ1)
        return dW1, db1, dW2, db2

    def update_params(self,W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
        W1 = W1 - alpha * dW1
        b1 = b1 - alpha * db1    
        W2 = W2 - alpha * dW2  
        b2 = b2 - alpha * db2    
        return W1, b1, W2, b2
    
    def get_predictions(self,A2):
        return np.argmax(A2, 0)

    def get_accuracy(self,predictions, Y):
        print(predictions, Y)
        return np.sum(predictions == Y) / Y.size

    def gradient_descent(self,X, Y, alpha, iterations):
        W1, b1, W2, b2 = self.init_params()
        for i in range(iterations):
            Z1, A1, Z2, A2 = self.forward_prop(W1, b1, W2, b2, X)
            dW1, db1, dW2, db2 = self.backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
            W1, b1, W2, b2 = self.update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
            if i % 100 == 0:
                print("Iteration: ", i)
                predictions = self.get_predictions(A2)
                print(self.get_accuracy(predictions[0:20], Y[0:20]))
        return W1, b1, W2, b2
    
    def make_predictions(self, X, W1, b1, W2, b2):
        _, _, _, A2 = self.forward_prop(W1, b1, W2, b2, X)
        predictions = self.get_predictions(A2)
        return predictions

    def train(self, X_train: np.ndarray, y_train: np.ndarray):
        """Train the classifier.
        Use the perceptron update rule as introduced in Lecture 3.
        Parameters:
            X_train: a number array of shape (N, D) containing training data;
                N examples with D dimensions
            y_train: a numpy array of shape (N,) containing training labels
        """
        N, D = X_train.shape
        self.w = np.random.randn(self.n_class, D)

        ###### YOUR CODE STARTS HERE ######
        
        
        ###### YOUR CODE ENDS HERE ######
        pass

    def predict(self, X_test: np.ndarray) -> np.ndarray:
        """Use the trained weights to predict labels for test data points.
        Parameters:
            X_test: a numpy array of shape (N, D) containing testing data;
                N examples with D dimensions
        Returns:
            predicted labels for the data in X_test; a 1-dimensional array of
                length N, where each element is an integer giving the predicted
                class.
        """
        N, D = X_test.shape
        y_test = np.zeros(N)
        ###### YOUR CODE STARTS HERE ######
        
        
        ###### YOUR CODE ENDS HERE ######
        return y_test

In [119]:
lr = 0.10
n_epochs = 500

percept_CIFAR = Perceptron_dl(n_class_CIFAR, lr, n_epochs)
percept_CIFAR.train(X_train_CIFAR, y_train_CIFAR)

In [120]:
#W1, b1, W2, b2 = percept_CIFAR.init_params()
W1, b1, W2, b2 = percept_CIFAR.gradient_descent(X_train_CIFAR, y_train_CIFAR, 0.1, 300)

TypeError: sigmoid() missing 1 required positional argument: 'x'

In [121]:
pred_percept = percept_CIFAR.make_predictions(X_train_CIFAR, W1, b1, W2,b2)
print('The training accuracy is given by: %f' % (percept_CIFAR.get_accuracy(pred_percept, y_train_CIFAR)))

[5 8 8 ... 5 1 8] [6 9 9 ... 6 2 8]
The training accuracy is given by: 0.210875


### Validate Perceptron on CIFAR

In [122]:
#pred_percept = percept_CIFAR.predict(X_val_CIFAR)
pred_percept = percept_CIFAR.make_predictions(X_val_CIFAR, W1, b1, W2,b2)
print('The validation accuracy is given by: %f' % (percept_CIFAR.get_accuracy(pred_percept, y_val_CIFAR)))

[0 1 5 ... 8 8 0] [1 8 5 ... 9 1 1]
The validation accuracy is given by: 0.211300


### Test Perceptron on CIFAR

In [123]:
#pred_percept = percept_CIFAR.predict(X_test_CIFAR)
pred_percept = percept_CIFAR.make_predictions(X_test_CIFAR, W1, b1, W2,b2)
print('The testing accuracy is given by: %f' % (percept_CIFAR.get_accuracy(pred_percept, y_test_CIFAR)))

[1 8 8 ... 5 5 1] [3 8 8 ... 5 1 7]
The testing accuracy is given by: 0.210500


In [124]:
output_submission_csv('output/Logistic_submission_CIFAR.csv', percept_CIFAR.make_predictions(X_test_CIFAR, W1, b1, W2,b2))

# Logistic Classifier

The Logistic Classifier has 2 hyperparameters that you can experiment with:
- **Learning rate** - similar to as defined above in Perceptron, this parameter scales by how much the weights are changed according to the calculated gradient update. 
- **Number of Epochs** - As described for perceptron.




The following code: 
- Creates an instance of the Logistic classifier class 
- The train function of the Logistic class is trained on the training data
- We use the predict function to find the training accuracy as well as the testing accuracy

In [125]:
"""Logistic regression model."""

import numpy as np
import math


class Logistic:
    def __init__(self, lr: float, epochs: int):
        """Initialize a new classifier.
        Parameters:
            lr: the learning rate
            epochs: the number of epochs to train for
        """
        self.w = None
        self.lr = lr
        self.epochs = epochs
        self.threshold = 0.5
        
    def sigmoid(self, x: np.ndarray) -> np.ndarray:
        """Sigmoid function.
        Parameters:
            z: the input
        Returns:
            the sigmoid of the input
        """
        ##### YOUR CODE STARTS HERE #####
        # x = x / np.linalg.norm(x, axis=-1)[:, np.newaxis]
        x = x / np.linalg.norm(x, axis=-1)[:, np.newaxis]
        return 1 / (1 + np.exp(-x))
        
        ##### YOUR CODE ENDS HERE #####
        
    def train(self, X_train: np.ndarray, y_train: np.ndarray, epochs, lr):
        """Train the classifier.
        Use the logistic regression update rule as introduced in lecture.
        Parameters:
            X_train: a numpy array of shape (N, D) containing training data;
                N examples with D dimensions
            y_train: a numpy array of shape (N,) containing training labels
        """
        D, N = X_train.shape
        self.w = np.random.randn(1, D)
        self.bias = 0
        
        #GD
        for _ in range(self.epochs):
            linear_model = np.dot(self.w, X_train) + self.bias
            y_predicted = self.sigmoid(linear_model)
            
           # self.predict(X_train, y_train, self.threshold)
            
            dw = (1/N) * np.dot((y_predicted - y_train), X_train.T)
            db = (1/N) * np.sum(y_predicted - y_train)
            
            self.w -= self.lr * dw
            self.bias -= self.lr * db

        ##### YOUR CODE STARTS HERE #####
        
        
        ##### YOUR CODE ENDS HERE #####
        pass

    def predict(self, X_test: np.ndarray) -> np.ndarray:
        """Use the trained weights to predict labels for test data points.
        Parameters:
            X_test: a numpy array of shape (N, D) containing testing data;
                N examples with D dimensions
        Returns:
            predicted labels for the data in X_test; a 1-dimensional array of
                length N, where each element is an integer giving the predicted
                class.
        """
        N, D = X_test.shape
        y_test = np.zeros(N)
        ##### YOUR CODE STARTS HERE #####
        
        linear_model = np.dot(self.w , X_test) + self.bias
        y_predicted = self.sigmoid(linear_model)
        
        y_predicted = y_predicted.T
        
        print(y_predicted.shape)
        
        y_predicted_class = [1 if i > self.threshold else 0 for i in y_predicted]
        
       # print(self.get_accuracy(y_predicted_class, y_train))
        
        
        
        ##### YOUR CODE ENDS HERE #####
        return y_predicted_class
    
    def get_accuracy(self,predictions, Y):
        print(predictions[0:20], Y[0:20])
        return np.sum(predictions == Y) / Y.size

### Training Logistic Classifer

In [126]:
#y_train_CIFAR[0:20]
y_train_CIFAR = np.where(y_train_CIFAR == 9, 1, 0)
#y_train_CIFAR[0:20]

In [127]:
learning_rate = 0.1
n_epochs = 1000

lr = Logistic(learning_rate, n_epochs)
lr.train(X_train_CIFAR, y_train_CIFAR,learning_rate, n_epochs)

In [128]:
pred_lr = lr.predict(X_train_CIFAR)
print('The training accuracy is given by: %f' % (lr.get_accuracy(pred_lr, y_train_CIFAR)))

(40000, 1)
[0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0] [0 1 1 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0]
The training accuracy is given by: 0.587700


### Validate Logistic Classifer

In [129]:
pred_lr = lr.predict(X_val_CIFAR)
print('The validation accuracy is given by: %f' % (get_acc(pred_lr, y_val_CIFAR)))

(10000, 1)
The validation accuracy is given by: 8.950000


### Test Logistic Classifier

In [131]:
pred_lr = lr.predict(X_test_CIFAR)
print('The testing accuracy is given by: %f' % (get_acc(pred_lr, y_test_CIFAR)))
output_submission_csv('output/Logistic_submission_CIFAR.csv', lr.predict(X_test_CIFAR))

(10000, 1)
The testing accuracy is given by: 8.530000
(10000, 1)
